# Classify organs present in ultrasound images

In [ ]:
import os
import numpy
from fastai.vision import *

In [ ]:
# Indicate the location of the pretrained torchvision models 
os.environ['TORCH_HOME'] = Path('models/resnet').absolute().as_posix()

## Data import

In [ ]:
valid_frac = 0.2
data_path = "data/Organ_Liver_Others/"
data = ImageDataBunch.from_folder(data_path, valid_pct=valid_frac)
#data = ImageDataBunch.from_folder(data_path, valid_pct=valid_frac,tfms=transforms())

## Model initial training

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=AUROC())
#learn = cnn_learner(data, models.resnet34, metrics=[accuracy, auc_roc_score])

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 3e-3 #get lr from plot

In [ ]:
callbacks_opt = [callbacks.SaveModelCallback(learn,
                                             monitor='AUROC()',
                                             every='improvement',
                                             name='best_B_AUROC_resnet34'),
                callbacks.EarlyStoppingCallback(learn, monitor='AUROC()', min_delta=.01, patience=3)]
#If "AUROC()" is the wrong statement, change it to "auc_roc_score"

In [ ]:
learn.fit(10, lr, callbacks=callbacks_opt)

We select the best model from the early stopping

In [ ]:
learn.load('best_B_AUROC_resnet34');

In [ ]:
learn.summary()

### Check the confusion matrix to identify problematic classes

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(normalize=False, figsize=(8, 8))

## Fine tuning of the model

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
callbacks_opt = [callbacks.SaveModelCallback(learn,
                                             monitor='AUROC()',
                                             every='improvement',
                                             name='best_B_AUROC_unfreeze_resnet34'),
                callbacks.EarlyStoppingCallback(learn, monitor='AUROC()', min_delta=.003, patience=3)]

In [ ]:
lr_max = 1e-4 

In [ ]:
learn.fit(10, slice(lr_max/5, lr_max), callbacks=callbacks_opt)

In [ ]:
interp_02 = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp_02.plot_confusion_matrix(normalize=False, figsize=(8, 8))

In [ ]:
interp_02.plot_top_losses(9)

In [ ]:
interp_02.plot_multi_top_losses()

In [ ]:
#freeze bottom layer
learn.freeze()

In [ ]:
learn.summary()

## Prediction

In [ ]:
img = Image.open(path)
learn.predict(img)